In [3]:
###
from typing import List

from netqasm.sdk.classical_communication.socket import Socket
from netqasm.sdk.connection import BaseNetQASMConnection
from netqasm.sdk.epr_socket import EPRSocket
from netqasm.sdk.qubit import Qubit

from squidasm.run.stack.config import GenericQDeviceConfig, LinkConfig, StackConfig, StackNetworkConfig
from squidasm.sim.stack.program import Program, ProgramContext, ProgramMeta
from squidasm.run.stack.run import run
from squidasm.util import create_two_node_network
###

import random

class Alice(Program):
    PEER_NAME = 'Bob'

    def __init__(self):
        super().__init__()

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name='Alice',
            csockets=[self.PEER_NAME],
            epr_sockets=[self.PEER_NAME]
            ###
            ,
            max_qubits=2,
            ###
        )

    def run(self, context: ProgramContext):
        connection = context.connection
        epr_socket = context.epr_sockets[self.PEER_NAME]
        csocket = context.csockets[self.PEER_NAME]

        bases = []
        key_bits = []
        for _ in range(10):  # Example for 10 bits of key
            # Create EPR pair
            epr = epr_socket.create_keep()[0]
            yield from connection.flush()

            # Decide randomly on a basis to measure, 0 for Z, 1 for X
            basis = random.randint(0, 1)
            bases.append(basis)

            # Measure based on chosen basis
            if basis == 0:
                epr.measure()  # Z-basis
            else:
                epr.H()  # Switch to X-basis before measurement
                epr.measure()

            yield from connection.flush()

        # After all measurements, send basis choices to Bob
        for basis in bases:
            csocket.send(str(basis))
        yield from connection.flush()

class Bob(Program):
    PEER_NAME = 'Alice'

    def __init__(self):
        super().__init__()

    @property
    def meta(self) -> ProgramMeta:
        return ProgramMeta(
            name='Bob',
            csockets=[self.PEER_NAME],
            epr_sockets=[self.PEER_NAME]
            ###
            ,
            max_qubits=2,
            ###
        )

    def run(self, context: ProgramContext):
        connection = context.connection
        epr_socket = context.epr_sockets[self.PEER_NAME]
        csocket = context.csockets[self.PEER_NAME]

        bases = []
        key_bits = []
        for _ in range(10):
            # Receive EPR pair
            epr = epr_socket.recv_keep()[0]
            yield from connection.flush()

            # Decide randomly on a basis to measure, 0 for Z, 1 for X
            basis = random.randint(0, 1)
            bases.append(basis)

            # Measure based on chosen basis
            if basis == 0:
                epr.measure()  # Z-basis
            else:
                epr.H()  # Switch to X-basis before measurement
                epr.measure()

            yield from connection.flush()

        # Receive basis choices from Alice
        alice_bases = []
        for _ in range(10):
            alice_basis = yield from csocket.recv()
            alice_bases.append(int(alice_basis))

        # Compare basis choices and keep measurements where bases match
        matching_bases = [i for i, basis in enumerate(bases) if basis == alice_bases[i]]
        # Key is formed from bits where Alice and Bob's bases matched

# The code above is simplified and conceptual.

###
run(
    config=create_two_node_network(node_names=['Alice', 'Bob'], link_noise=0),
    programs={
        'Alice': Alice(),
        'Bob': Bob(),
    },
    num_times=1,
)
###

[[None], [None]]